In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [5]:
path = '~/data/tinkoff/'

cust = pd.read_csv(path + 'customer_test.csv')
transactions = pd.read_csv(path + 'transactions.csv')
react_train = pd.read_csv(path + 'stories_reaction_train.csv')
react_test = pd.read_csv(path + 'stories_reaction_test.csv')
descr = pd.read_csv(path + 'stories_description.csv')
sample = pd.read_csv(path + 'sample_submit.csv')

In [6]:
def preparation(data):
  data = data.set_index('customer_id')
  
  # OneHot for products
  for i in range(7):
    column = 'product_{}'.format(i)
    for status in {'UTL', 'CLS', 'OPN'}:
      data['{}_{}'.format(column, status)] = (data[column] == status).astype(int)
    data['{}_{}'.format(column, 'nan')] = (data[column] != data[column]).astype(int)
    data.drop(column, inplace=True, axis = 1)
  
  # Male = 1, Female = 0
  data['gender_cd'] = (data['gender_cd'] == 'M').astype(int)
  
  # job_title lower
  data['job_title'] = data['job_title'].str.lower()
  
  # n_most_common job_titles to OneHot
  n_most_common = 13
  data['job_nan_or_rare'] = (data['age'] == -1).astype(int)
  for title, freq in zip(data['job_title'].value_counts()[:n_most_common + 1].index, range(n_most_common + 1)):
    # print(title, freq)
    data['{}_{}'.format('job', str(freq))] = (data['job_title'] == title).astype(int)
    data['job_nan_or_rare'] = data['job_nan_or_rare'] + (data['job_title'] == title).astype(int)
  data['job_nan_or_rare'] = (data['job_nan_or_rare'] == 0).astype(int)
  data.drop('job_title', inplace=True, axis=1)
  data.drop('job_position_cd', inplace=True, axis=1)
  
  # first_session to months since first member divided by coef to get OneHot.
  coef = 6
  data['first_session_dttm'] = data['first_session_dttm'].str.split('-').apply(lambda x: int(x[0]) * 12 + int(x[1]) if x == x else 'nan')
  notnan = data['first_session_dttm'] != 'nan'
  first = data['first_session_dttm'][notnan].min()
  data['first_session_dttm'][notnan] = data['first_session_dttm'][notnan].apply(lambda x: (x - first) // coef)
  
  # first_session to OneHot
  for status in data['first_session_dttm'].unique():
    if status != status:
      data['first_session_nan'] = (data['first_session_dttm'] != data['first_session_dttm']).astype(int)
    else:
      data['first_session_{}'.format(status)] = (data['first_session_dttm'] == status).astype(int)
  data.drop('first_session_dttm', inplace=True, axis=1)
  
  # marital_status to OneHot
  for status in data['marital_status_cd'].unique():
    if status != status:
      data['marital_status_nan'] = (data['marital_status_cd'] != data['marital_status_cd']).astype(int)
    else:
      data['marital_status_{}'.format(status)] = (data['marital_status_cd'] == status).astype(int)
  data.drop('marital_status_cd', inplace=True, axis=1)
  return data

new_data = preparation(cust)
print('Признаков =', len(new_data.columns))
new_data.head(6)

Признаков = 69


/home/data_sapiens/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,gender_cd,age,children_cnt,product_0_UTL,product_0_CLS,product_0_OPN,product_0_nan,product_1_UTL,product_1_CLS,product_1_OPN,...,first_session_13.0,first_session_nan,first_session_14.0,marital_status_nan,marital_status_UNM,marital_status_CIV,marital_status_MAR,marital_status_DIV,marital_status_WID,marital_status_DLW
customer_id,,,,,,,,,,,,,,,,,,,,,
234305,0,25.0,0.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
914339,1,15.0,0.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
895631,1,20.0,0.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
954837,0,25.0,0.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
391590,1,30.0,0.0,0,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
385518,1,30.0,0.0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [53]:
train.shape

(473141, 70)

In [142]:
train = react_train.join(new_data, on = 'customer_id')
test = react_test.join(new_data, on = 'customer_id')

In [143]:
test['answer_id'].head()

0    0
1    1
2    2
3    3
4    4
Name: answer_id, dtype: int64

In [144]:
train.drop('event_dttm', axis = 1, inplace = True)
test.drop('event_dttm', axis=1, inplace=True)

In [13]:
from sklearn.preprocessing import LabelEncoder

In [145]:
train.drop('customer_id', axis = 1, inplace = True)
test.drop('customer_id', axis=1, inplace=True)

In [102]:
y = LabelEncoder().fit_transform(train2['event'])

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

In [101]:
model = Sequential()
model.add(InputLayer(69))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [103]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)

In [104]:
model.fit(train2.drop('event', axis = 1), y, validation_split=0.5, epochs=2)

Train on 982166 samples, validate on 982167 samples
Epoch 1/2
982166/982166 [==============================] - 285s 291us/sample - loss: 0.6929 - acc: 0.5267 - val_loss: 9.8163 - val_acc: 0.0043
Epoch 2/2
982166/982166 [==============================] - 227s 231us/sample - loss: 0.6891 - acc: 0.5364 - val_loss: 10.2998 - val_acc: 0.0047


In [148]:
train['age'].fillna(train['age'].median(), inplace=True)
train['children_cnt'].fillna(train['children_cnt'].median(), inplace=True)
test['age'].fillna(train['age'].median(), inplace=True)
test['children_cnt'].fillna(train['children_cnt'].median(), inplace=True)
# test.isna().any()

In [65]:
ar = model.predict(test.values)

In [72]:
ar[ar[:, 0] > 0.3]

array([[0.39186677, 0.1914524 , 0.21574314, 0.20093773],
       [0.39186677, 0.1914524 , 0.21574314, 0.20093773],
       [0.39186677, 0.1914524 , 0.21574314, 0.20093773],
       [0.39186677, 0.1914524 , 0.21574314, 0.20093773],
       [0.39186677, 0.1914524 , 0.21574314, 0.20093773],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253784, 0.19601496, 0.22589216, 0.19555512],
       [0.38253784, 0.19601496, 0.22589216, 0.19555512],
       [0.38253787, 0.19601493, 0.22589213, 0.19555509],
       [0.38253787, 0.19601493,

In [82]:
train['event'].value_counts()

view       242312
skip       167567
like        56687
dislike      6575
Name: event, dtype: int64

In [83]:
167567 / 6575

25.485475285171102

In [84]:
167567 / 56687

2.9560040220861925

In [87]:
likes = train[train['event'] == 'like']
dislikes = train[train['event'] == 'dislike']
skips = train[train['event'] == 'skip']
views = train[train['event'] == 'view']

In [98]:
train2 = pd.concat([views] * 2 + [skips] * 3 + [likes] * 9 + [dislikes] * 71)

In [99]:
train2['event'].value_counts()

like       510183
skip       502701
view       484624
dislike    466825
Name: event, dtype: int64

In [ ]:
if dislike > 0.5:
    -1
else:
    1

In [106]:
y_test = model.predict(test)

In [107]:
sample.head()

,answer_id,score
0,0,0.85
1,1,0.53
2,2,-0.89
3,3,0.54
4,4,0.34


In [108]:
react_test.head()

,customer_id,story_id,event_dttm,answer_id
0,466906,1152,2018-08-01 00:00:46,0
1,466906,1355,2018-08-01 00:00:46,1
2,146395,537,2018-08-01 00:02:08,2
3,897303,915,2018-08-01 00:02:30,3
4,50094,1415,2018-08-01 00:03:13,4


In [118]:
print(y_test[y_test[:, 0] > 0.001])

[[0.00183112 0.00118643 0.52823    0.46875256]
 [0.00112576 0.0007622  0.52870905 0.469403  ]
 [0.00112576 0.0007622  0.52870905 0.469403  ]
 ...
 [0.00124206 0.00083154 0.53368634 0.4642401 ]
 [0.00124206 0.00083154 0.53368634 0.4642401 ]
 [0.00124206 0.00083154 0.53368634 0.4642401 ]]


In [119]:
y_test[:4]

array([[9.1747388e-05, 7.9007368e-05, 5.2982575e-01, 4.7000343e-01],
       [9.1747388e-05, 7.9007368e-05, 5.2982575e-01, 4.7000343e-01],
       [2.3250612e-04, 1.8183373e-04, 5.8355933e-01, 4.1602635e-01],
       [8.2050501e-05, 6.0725804e-05, 5.9720862e-01, 4.0264863e-01]],
      dtype=float32)

In [120]:
sample.head()

,answer_id,score
0,0,0.85
1,1,0.53
2,2,-0.89
3,3,0.54
4,4,0.34


In [121]:
sample['score'] = -1

In [122]:
sample.to_csv('name.csv', index = False)

In [131]:
tr = react_train['event'][:]
for status, num in zip(react_train['event'].unique(), range(4)):
    tr[tr == status] = num
    print(status, num)
t = to_categorical(tr) 


0 0
1 1
2 2
3 3


In [136]:
print(train2['event'].value_counts())
for i in range(4):
    print(sum(y[:,i]))

like       510183
skip       502701
view       484624
dislike    466825
Name: event, dtype: int64
466825.0
510183.0
502701.0
484624.0


1 - dislike
2 - like
3 - skip
4 - view

In [151]:
def res(ar):
    ans = 2 * (ar[:, 1] + ar[:, 3] > 0.5) - 1
    return ans


In [140]:
sample.head()

,answer_id,score
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1


In [149]:
test.head()

,story_id,answer_id,gender_cd,age,children_cnt,product_0_UTL,product_0_CLS,product_0_OPN,product_0_nan,product_1_UTL,...,first_session_13.0,first_session_nan,first_session_14.0,marital_status_nan,marital_status_UNM,marital_status_CIV,marital_status_MAR,marital_status_DIV,marital_status_WID,marital_status_DLW
0,1152,0,0,25.0,0.0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1355,1,0,25.0,0.0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,537,2,1,25.0,0.0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
3,915,3,0,30.0,0.0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
4,1415,4,1,30.0,0.0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [152]:
sample['score'] = res(model.predict(test.drop(['answer_id', 'story_id'], axis = 1)))

In [153]:
sample.head(10)

,answer_id,score
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,-1


In [161]:
sample.to_csv('lasthope.csv', index = False)

In [155]:
train.head(1)

,story_id,event,gender_cd,age,children_cnt,product_0_UTL,product_0_CLS,product_0_OPN,product_0_nan,product_1_UTL,...,first_session_13.0,first_session_nan,first_session_14.0,marital_status_nan,marital_status_UNM,marital_status_CIV,marital_status_MAR,marital_status_DIV,marital_status_WID,marital_status_DLW
0,138,0,1,40.0,1.0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [156]:
check = res(model.predict(train.drop(['event', 'story_id'], axis = 1)))

array([-1, -1, -1, ..., -1, -1, -1])

In [160]:
check[:30]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [158]:
train['event'][:10]

0    0
1    1
2    1
3    1
4    0
5    0
6    0
7    0
8    1
9    0
Name: event, dtype: object